# Warehouse Problem - Benders

Let's recall the formulations.

The MIP model is:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \sum_{w ,c} t_{w,c} y_{w,c} + \sum_c M z_c & \\
\text{subject to:} &&\\
&y_{w,c} \leq x_w & \forall w,c \\
&\sum_w y_{w,c} + z_c = 1 & \forall c \\
&\sum_c y_{w,c} \leq C_w  x_w & \forall w \\
&x_w, y_{w,c}, z_c \in \mathbb{B} & \forall w,c
\end{align*}

The Benders decomposition is:

#### Master:

\begin{align*}
\min &\qquad \sum_w c_w x_w + \theta & \\
\text{subject to:} && \\
&\theta \geq \sum_w (C_w \gamma^n_w + \sum_c \alpha^n_{w,c}) x_w + \sum_c \beta^n_c & \forall n \\
&x_w \in \mathbb{B} & \forall w,c \\
&\theta \geq 0 &
\end{align*}

#### Subproblem:

\begin{align*}
\min & \qquad \sum_{w ,c} t_{w,c} y_{w,c} + \sum_c M z_c & \\
\text{subject to:} && \\
&y_{w,c} \leq x^n_w & \forall w,c \quad (\alpha_{w,c}) \\
&\sum_w y_{w,c} + z_c = 1 & \forall c \quad (\beta_c) \\
&\sum_c y_{w,c} \leq C_w  x^n_w & \forall w \quad (\gamma_w) \\
&y_{w,c}, z_c \geq 0 & \forall w,c
\end{align*}

In [1]:
fixed = 30  # c_w (all the costs are the same)
M = 10*fixed  # the penalty
capacity = [1,4,2,1,3]  # the capacity of a warehouse
supplyCost = [[20,24,11,25,30],  # t_{w,c}
              [28,27,82,83,74],
              [74,97,71,96,70],
              [2,55,73,69,61],
              [46,96,59,83,4],
              [42,22,29,67,59],
              [1,5,73,59,56],
              [10,73,13,43,96],
              [93,35,63,85,46],
              [47,65,55,71,95]]
nbStores = len(supplyCost)
nbWarehouses = len(capacity)
Stores = range(nbStores)
Warehouses = range(nbWarehouses)

### Excercices

#### 1. Implement the subproblem.

In [2]:
from docplex.mp.model import Model

def benders_subproblem(x_sol):
   
    mdl = Model(name='sub_problem')
    y = mdl.continuous_var_matrix(nbStores, nbWarehouses, name='supplier')
    z = mdl.continuous_var_list(nbStores, name='not_served')

    # collect constraints
    
    supplies = [ [ 0 for s in Stores] for w in Warehouses]
    stores = [ 0 for s in Stores ]
    opens = [ 0 for w in Warehouses]
    
    # state the constraints

    for w in Warehouses:
        for s in Stores:
            supplies[w][s] = mdl.add_constraint(y[s,w] <= x_sol[w], 'supply_%d_%d' % (s, w))
            
    for s in Stores:
        stores[s] = mdl.add_constraint(sum(y[s,w] for w in Warehouses) + z[s] >= 1, 'store_%d' % s)
        
    for w in Warehouses:
        opens[w] = mdl.add_constraint(sum(y[s,w] for s in Stores) <= capacity[w] * x_sol[w], 'open_%d' % w) 
        
    mdl.minimize(M * sum(z) + sum(supplyCost[s][w] * y[s,w]  for s in Stores for w in Warehouses))
   
    msol = mdl.solve()
    
    # duals of the second set of constraints
    const = sum(stores[s].dual_value for s in Stores)
    
    # duals of the first and third sets of constraints
    coeffs = [sum(supplies[w][s].dual_value for s in Stores) + capacity[w] * opens[w].dual_value for w in Warehouses]
    return (mdl.objective_value,  # the objective value
            const,  # the constant part of the benders' cut
            coeffs)  # the coefficients of the benders' cut

#### Try the function

In [3]:
benders_subproblem([0]*nbWarehouses)

(3000.0, 3000.0, [-2637.0, -2501.0, -2471.0, -2319.0, -2409.0])

#### 2. Implement the master problem

Create a master problem without any cut and solve it.

In [4]:
m_mdl = Model(name='master')

x = m_mdl.binary_var_list(nbWarehouses, name='open')
theta = m_mdl.continuous_var(lb=0, name='theta')
m_mdl.minimize(fixed * sum(x) + theta)
m_mdl.solve()
print('Solution cost: %.2f' % m_mdl.objective_value)

Solution cost: 0.00


#### Then, create the resolution loop that will:
1. Solve the master problem
2. Solve the sub-problem with a solution
3. Add the benders' cut to the master problem

In [7]:
# build the master
master = Model(name='master')
x = master.binary_var_list(nbWarehouses, name='open')
# to bound the objective of the subproblem
theta = master.continuous_var(lb=0, name='theta')
master.minimize(fixed * sum(x) + theta)

iteration = 0
LB = 0
UB = M * nbStores

while True:    
    # 1. Solve the master
    master.solve()
    LB = master.objective_value
    
    # 2. Solve the subproblem
    x_sol = [x[w].solution_value for w in Warehouses]
    sub_objective_value, const, coeffs = benders_subproblem(x_sol)

    # UB = min(UB, ub + fixed * sum(x_sol))
    UB = fixed * sum(x_sol) + sub_objective_value
    print('Iter=%d; LB=%.2f; UB=%.2f' %(iteration, LB, UB))
    
    # check if you have guess theta correctly
    if abs(LB - UB) < 1e-5:
        break
    
    # 3. Add the benders' cut to the master problem
    master.add_constraint(theta >= const + sum(coeffs[w] * x[w] for w in Warehouses),'bender_cut_%d' % iteration)
    iteration += 1

print(''.join(['=']*40))
for w in [w for w in Warehouses if x[w].solution_value > 0]:
    print('Warehouse %d is open.' % w)

Iter=0; LB=0.00; UB=3000.00
Iter=1; LB=60.00; UB=1640.00
Iter=2; LB=60.00; UB=1682.00
Iter=3; LB=60.00; UB=1983.00
Iter=4; LB=60.00; UB=1373.00
Iter=5; LB=60.00; UB=1144.00
Iter=6; LB=90.00; UB=1172.00
Iter=7; LB=90.00; UB=1463.00
Iter=8; LB=90.00; UB=638.00
Iter=9; LB=90.00; UB=917.00
Iter=10; LB=90.00; UB=884.00
Iter=11; LB=90.00; UB=1958.00
Iter=12; LB=90.00; UB=1105.00
Iter=13; LB=90.00; UB=1413.00
Iter=14; LB=90.00; UB=1406.00
Iter=15; LB=120.00; UB=437.00
Iter=16; LB=313.00; UB=904.00
Iter=17; LB=315.00; UB=383.00
Iter=18; LB=383.00; UB=383.00
Warehouse 0 is open.
Warehouse 1 is open.
Warehouse 2 is open.
Warehouse 4 is open.
